In [1]:
import gradio as gr
import torch
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import VectorDBQA
from langchain.document_loaders import TextLoader
from langchain.vectorstores import Chroma

from langchain.embeddings import HuggingFaceEmbeddings

from langchain.embeddings import HuggingFaceInstructEmbeddings

vectordb = None
hf = None
def update(input):
    global hf
    global vectordb

    model_select = 1
    model_names = ['sentence-transformers/multi-qa-mpnet-base-dot-v1', # 0
               'intfloat/multilingual-e5-large', # 1
               'shibing624/text2vec-base-chinese', # 2
               'hkunlp/instructor-large', # 3
               'GanymedeNil/text2vec-large-chinese', # 4
               'sentence-transformers/all-mpnet-base-v2', # 5
               'shibing624/text2vec-base-chinese-paraphrase', # 6
               ]
    if input == 'mpnet':
        model_select = 0
    elif input == 'e5':
        model_select = 1
    elif input == 'shibing':
        model_select = 2
    elif input == 'all_mpnet':
        model_select = 5
    elif input == 'ganymedeNil':
        model_select = 4
    elif input == 'shibing_paraphrase':
        model_select = 6

    model_name = model_names[model_select]

    if 'hf' in locals() or 'hf' in globals():
        print('clear gpu embedding memory')
        del hf
        torch.cuda.empty_cache()

    print(f'mode {model_name} is selected!')

    model_kwargs = {'device': 'cuda'}
    encode_kwargs = {'normalize_embeddings': True}
    hf = HuggingFaceEmbeddings(
        model_name=model_name,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs
    )

    model_name_pre = model_name.split('/')[-1]
    relative_path = '../'

    model_name_pre = model_name.split('/')[-1]
    parent_path = './test/'
    persist_directory = parent_path + 'survey_db_'+model_name_pre+'_embedding'
    persist_directory = relative_path + persist_directory


    vectordb = Chroma(persist_directory=persist_directory, embedding_function=hf)

    return f'load model {model_name} in {persist_directory}'
    
def greet(query):
    ans = vectordb.similarity_search_with_score(query, k=5)
    print(f'query is {query}')
    return ans


update('e5')

# with gr.Blocks() as demo:
#     with gr.Row():
#         a = gr.Interface(fn=greet, inputs="text", outputs="text")

with gr.Blocks() as demo:
    model_selected = gr.Dropdown(['mpnet','e5','shibing','all_mpnet','ganymedeNil','shibing_paraphrase'])
    refresh_button = gr.Button('refresh')
    refresh_button.click(fn=update,inputs=model_selected, outputs=gr.Textbox())
    with gr.Row():
        gr.Interface(fn=greet, inputs="text", outputs=["text",
                                                           "text",
                                                           "text",
                                                           "text",
                                                           "text",])
    
demo.launch(server_name="0.0.0.0",share=True)   

clear gpu embedding memory
mode intfloat/multilingual-e5-large is selected!


No sentence-transformers model found with name C:\Users\zhang/.cache\torch\sentence_transformers\intfloat_multilingual-e5-large. Creating a new one with MEAN pooling.


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Running on local URL:  http://0.0.0.0:7860
Running on public URL: https://78edfce1dee104683e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
vectordb.get(where={"name":"寒武纪"})

In [ ]:
vectordb.similarity_search_with_score('智能加速卡',k=1,filter={"name":"寒武纪"})